<a href="https://colab.research.google.com/github/chamithZ/FineTune-DeepSeek8B-CO2EmissionReducerModel/blob/main/FineTuneDeepSeekR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(  # wrap pre trained model with LoRA
    model,
    r = 8, #  8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
{"role": "system", "content": "You are a knowledgeable chatbot on reducing CO2 emissions from vehicles."}
{"role": "user", "content": "How can I reduce CO2 emissions from my vehicle?"}
{"role": "assistant", "content": "You can reduce CO2 emissions by maintaining proper tire pressure, driving smoothly, reducing unnecessary weight, and keeping your engine well-maintained."}


{'role': 'assistant',
 'content': 'You can reduce CO2 emissions by maintaining proper tire pressure, driving smoothly, reducing unnecessary weight, and keeping your engine well-maintained.'}

In [ ]:
import json
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
from datasets import Dataset

# Load the JSON file
file_path = '/content/Vehicle_Details/co2_reduction_qa.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# Group the messages into conversations (pair user and assistant messages)
conversations = []
current_convo = []
for message in data:
    current_convo.append(message)
    if message["role"] == "assistant":  # End of a conversation turn (user question + assistant answer)
        conversations.append(current_convo)
        current_convo = []

# Convert list of conversations into a Hugging Face Dataset
dataset = Dataset.from_dict({"conversations": conversations})

# Apply the standardize_sharegpt function
dataset = standardize_sharegpt(dataset)

# Define the tokenizer with a chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# Define the formatting function for the prompts
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Map over the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)


ModuleNotFoundError: No module named 'unsloth'

In [ ]:
dataset[5]["conversations"]

[{'content': 'How to reduce CO2 emissions from FORD New Focus Model Year Post 2024.00 vehicle?',
  'role': 'user'},
 {'content': 'Use the correct grade of motor oil recommended by the manufacturer.',
  'role': 'assistant'}]

In [ ]:
dataset[5]["text"]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Applying chat template to train dataset (num_proc=2):   0%|          | 0/357 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/357 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/357 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/357 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 357 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 220
 "-____-"     Number of trainable parameters = 20,971,520
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chamithbis (chamithbis-sliit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,5.131400
2,4.512300
3,4.742700
4,4.507900
5,3.974100
6,3.107300
7,2.781500
8,2.280200
9,1.695700
10,1.664400


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How many national parks in Sri Lanka?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

ModuleNotFoundError: No module named 'unsloth'

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "How to reduce CO2 emissions from a Honda Civic?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Drive at moderate speeds to optimize fuel consumption.<|eot_id|><|eot_id|>

Avoid aggressive acceleration and braking to reduce rolling resistance.<|eot_id|><|eot_id|>

Use the correct grade of motor oil recommended by the manufacturer.<|eot_id|><|eot_id|>

Check and maintain proper tire pressure regularly.<|eot_id|><|eot_id|>

Plan routes efficiently to minimize unnecessary driving.<|eot_id|><|eot_id|>

Turn off the engine when idling for long periods.<|eot_id|><|eot_id|>

Use cruise control on highways to maintain a steady speed.<|eot_id|><|eot_id|>

Upgrade to energy-efficient spark plugs for better combustion.<|eot_id|><|eot_id|>

Reduce unnecessary weight in the trunk to reduce fuel consumption.<|eot_id|><|eot_id|>

Avoid excessive air conditioning use to improve fuel economy.<|eot_id|>


In [ ]:
model.save_pretrained_gguf("DeepSeekFineTuned_CO2EmissionReducerModel", tokenizer, quantization_method = "q8_0")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.77 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 14.02it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:53<00:00,  5.42s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving DeepSeekFineTuned_CO2EmissionReducerModel/pytorch_model-00001-of-00004.bin...
Unsloth: Saving DeepSeekFineTuned_CO2EmissionReducerModel/pytorch_model-00002-of-00004.bin...
Unsloth: Saving DeepSeekFineTuned_CO2EmissionReducerModel/pytorch_model-00003-of-00004.bin...
Unsloth: Saving DeepSeekFineTuned_CO2EmissionReducerModel/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at DeepSeekFineTuned_CO2EmissionReducerModel into q8_0 GGUF format.
The output location will be /content/DeepSeekFineTuned_CO2EmissionReducerModel/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: DeepSeekFineTuned_CO2EmissionReducerModel
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map fr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_path = "/content/drive/MyDrive/DeepSeekFineTuned_CO2EmissionReducerModel"


In [ ]:
model.save_pretrained_gguf(
    save_path,
    tokenizer,
    quantization_method = "q8_0"
)


NameError: name 'model' is not defined